# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [1]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [2]:
# Initialization

load_dotenv()

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

# As an alternative, if you'd like to use Ollama instead of OpenAI
# Check that Ollama is running for you locally (see week1/day2 exercise) then uncomment these next 2 lines
# MODEL = "llama3.2"
# openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')


OpenAI API Key exists and begins sk-proj-


In [3]:
system_prompt = "You are a technical expert of all coding languages as well as maths and statistics.\
      Using your expertise you must answer a questions the user has"

In [4]:
# Get gpt-4o-mini to answer, with streaming
def answer_gpt(question, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": question}]
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=messages,
        stream=True
    )
    
    response = ""
    for chunk in stream:
        response+= chunk.choices[0].delta.content or ''
        yield response
        

# Only chat-gpt. adding 
* groq, 
* gemini, and 
* ollama

In [5]:
gr.ChatInterface(fn=answer_gpt, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [6]:
# Get Llama 3.2 to answer
import ollama

MODEL = "llama3.2"
def answer_alpaca(question, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": question}]
    stream = ollama.chat(model=MODEL, messages=messages, stream=True)
    response = ""
    for chunk in stream:
        response+= chunk['message']['content'] or ''
        yield response


In [7]:
gr.ChatInterface(fn=answer_alpaca, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


In [8]:
import google.generativeai

In [9]:
google.generativeai.configure()

In [10]:
google_api_key = os.getenv('GOOGLE_API_KEY')

In [11]:
gemini_via_openai_client = OpenAI(
    api_key=google_api_key, 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [12]:
def answer_google(question, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": question}]
    stream = gemini_via_openai_client.chat.completions.create(
        model="gemini-1.5-flash",
        messages=messages,
        stream=True
    )
    response = ""
    for chunk in stream:
        response+= chunk.choices[0].delta.content or ''
        yield response

In [13]:
model_choice = gr.Radio(
    choices=["Llama", "Google", "GPT"],
    label="Choose AI Model",
    value="Llama"
)

def answer_with_model(question, history, model):
    # Convert the generator to a string before returning
    response = ""
    generator = None
    
    if model == "Llama":
        generator = answer_alpaca(question, history)
    elif model == "Google":
        generator = answer_google(question, history)
    else:
        generator = answer_gpt(question, history)
    
    # Consume the generator to get the final response
    for chunk in generator:
        response = chunk  # Each chunk is the cumulative response
    
    return response

gr.ChatInterface(
    fn=answer_with_model,
    chatbot=gr.Chatbot(),
    additional_inputs=[model_choice]
).launch()

c:\Users\tejas\Anaconda3\envs\llms\Lib\site-packages\gradio\components\chatbot.py:242: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(
c:\Users\tejas\Anaconda3\envs\llms\Lib\site-packages\gradio\chat_interface.py:229: UserWarning: The gr.ChatInterface was not provided with a type, so the type of the gr.Chatbot, 'tuples', will be used.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.
